In [ ]:
import numpy as np
from pomegranate import *

study_time = Node(DiscreteDistribution({
    'low': 0.2,
    'medium': 0.5,
    'high': 0.3
}), name='study_time')

exam_score = Node(ConditionalProbabilityTable([
    ['low', 'poor', 0.8],
    ['low', 'average', 0.15],
    ['low', 'good', 0.05],
    ['medium', 'poor', 0.4],
    ['medium', 'average', 0.4],
    ['medium', 'good', 0.2],
    ['high', 'poor', 0.1],
    ['high', 'average', 0.3],
    ['high', 'good', 0.6]
], [study_time.distribution]), name='exam_score')

pass_exam = Node(ConditionalProbabilityTable([
    ['poor', 'fail', 0.9],
    ['poor', 'pass', 0.1],
    ['average', 'fail', 0.5],
    ['average', 'pass', 0.5],
    ['good', 'fail', 0.2],
    ['good', 'pass', 0.8]
], [exam_score.distribution]), name='pass_exam')

model = BayesianNetwork('Student Exam Model')
model.add_states(study_time, exam_score, pass_exam)
model.add_edge(study_time, exam_score)
model.add_edge(exam_score, pass_exam)
model.bake()

prob_score_given_study_time = model.predict_proba({'study_time': 'medium'})[1]
print(f"Probability of exam score being 'average' given medium study time: {prob_score_given_study_time.parameters[0][1]:.2f}")

prob_pass_exam_given_score_and_study_time = model.predict_proba({'exam_score': 'average', 'study_time': 'medium'})[2]
print(f"Probability of passing the final exam given 'average' exam score and medium study time: {prob_pass_exam_given_score_and_study_time.parameters[0][1]:.2f}")


In [ ]:
# Define the joint probability table
joint_prob_table = {
    ('<40', 'No', 'High'): 100,
    ('<40', 'Yes', 'High'): 150,
    ('40-60', 'No', 'High'): 150,
    ('40-60', 'Yes', 'High'): 255,
    ('>60', 'No', 'High'): 150,
    ('>60', 'Yes', 'High'): 250,
    ('<40', 'No', 'Low'): 0,
    ('<40', 'Yes', 'Low'): 0,
    ('40-60', 'No', 'Low'): 0,
    ('40-60', 'Yes', 'Low'): 0,
    ('>60', 'No', 'Low'): 0,
    ('>60', 'Yes', 'Low'): 0
}

# Compute the total number of patients
total_patients = 1000

# Compute the total number of patients with heart disease
total_heart_disease_patients = 400

# Compute the prior probability of heart disease
prior_heart_disease = total_heart_disease_patients / total_patients

# Compute the marginal probability of age and cholesterol level
marginal_age_high_cholesterol = (
    joint_prob_table[('<40', 'Yes', 'High')] +
    joint_prob_table[('40-60', 'Yes', 'High')] +
    joint_prob_table [('>60', 'Yes', 'High')]
) / total_patients

# Compute the conditional probability of heart disease given age and high cholesterol
age = '40-60'
cholesterol = 'High'
conditional_heart_disease = (
    joint_prob_table[(age, 'Yes', cholesterol)] / total_heart_disease_patients *
    prior_heart_disease / marginal_age_high_cholesterol
)

print(f'Conditional probability of heart disease given age={age} and high cholesterol={cholesterol}: {conditional_heart_disease}')


In [ ]:
from pomegranate import *

# Define the nodes
cpu_usage = DiscreteDistribution({'low': 0.3, 'med': 0.4, 'high': 0.3})
memory_usage = DiscreteDistribution({'low': 0.4, 'med': 0.4, 'high': 0.2})
disk_usage = DiscreteDistribution({'low': 0.2, 'med': 0.4, 'high': 0.4})
network_traffic = DiscreteDistribution({'low': 0.4, 'med': 0.4, 'high': 0.2})
operating_system = DiscreteDistribution({'Windows': 0.6, 'Linux': 0.4})
app_type = ConditionalProbabilityTable([
    ['low', 'Windows', 'web', 0.3],
    ['med', 'Windows', 'web', 0.5],
    ['high', 'Windows', 'web', 0.2],
    ['low', 'Linux', 'web', 0.2],
    ['med', 'Linux', 'web', 0.3],
    ['high', 'Linux', 'web', 0.5],
    ['low', 'Windows', 'database', 0.2],
    ['med', 'Windows', 'database', 0.4],
    ['high', 'Windows', 'database', 0.4],
    ['low', 'Linux', 'database', 0.4],
    ['med', 'Linux', 'database', 0.4],
    ['high', 'Linux', 'database', 0.2],
], [cpu_usage, memory_usage, disk_usage, operating_system])

# Create the nodes and edges
cpu_node = Node(cpu_usage, name="CPU Usage")
memory_node = Node(memory_usage, name="Memory Usage")
disk_node = Node(disk_usage, name="Disk Usage")
network_node = Node(network_traffic, name="Network Traffic")
os_node = Node(operating_system, name="Operating System")
app_node = Node(app_type, name="Application Type")

model = BayesianNetwork("System Failure")
model.add_states(cpu_node, memory_node, disk_node, network_node, os_node, app_node)
model.add_edge(cpu_node, app_node)
model.add_edge(memory_node, app_node)
model.add_edge(disk_node, app_node)
model.add_edge(network_node, app_node)
model.add_edge(os_node, app_node)

# Finalize the model and compute the probability/likelihood of a system failure
model.bake()
prob_failure = model.predict_proba({'CPU Usage': 'high', 'Memory Usage': 'med', 'Disk Usage': 'low',
                                     'Network Traffic': 'med', 'Operating System': 'Windows'})[app_node].parameters[0]['database']
print("Probability of System Failure: {:.2f}%".format(prob_failure*100))


In [ ]:
# !pip install pgmpy
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

model = BayesianModel([('C', 'H'),('A','H')]) 

cpd_C=TabularCPD(variable='C',variable_card=4,values=[[0.2],[0.2],[0.3],[0.3]])
cpd_A=TabularCPD(variable='A',variable_card=3,values=[[0.2],[0.2],[0.6]])
cpd_H = TabularCPD(variable='H', variable_card=2, 
                   values=[[0.1, 0.2, 0.3, 0.4, 0.2, 0.4, 0.6, 0.8, 0.3, 0.6, 0.9, 1.0],
                           [0.9, 0.8, 0.7, 0.6, 0.8, 0.6, 0.4, 0.2, 0.7, 0.4, 0.1, 0.0]],
                   evidence=['A', 'C'], evidence_card=[3, 4])
model.add_cpds(cpd_H, cpd_A,cpd_C)

print(model.get_cpds('H'))
from pgmpy.inference import VariableElimination

# Create an instance of the VariableElimination class with the model
inference = VariableElimination(model)
print(inference.query(['H'], evidence={'A': 0}))
print(inference.map_query(variables=['A']))


+------+------+------+------+------+-----+------+------+------+------+------+
| A    | A(0) | A(0) | A(0) | A(0) | ... | A(1) | A(2) | A(2) | A(2) | A(2) |
+------+------+------+------+------+-----+------+------+------+------+------+
| C    | C(0) | C(1) | C(2) | C(3) | ... | C(3) | C(0) | C(1) | C(2) | C(3) |
+------+------+------+------+------+-----+------+------+------+------+------+
| H(0) | 0.1  | 0.2  | 0.3  | 0.4  | ... | 0.8  | 0.3  | 0.6  | 0.9  | 1.0  |
+------+------+------+------+------+-----+------+------+------+------+------+
| H(1) | 0.9  | 0.8  | 0.7  | 0.6  | ... | 0.2  | 0.7  | 0.4  | 0.1  | 0.0  |
+------+------+------+------+------+-----+------+------+------+------+------+
+------+----------+
| H    |   phi(H) |
+======+==========+
| H(0) |   0.2700 |
+------+----------+
| H(1) |   0.7300 |
+------+----------+


0it [00:00, ?it/s]

0it [00:00, ?it/s]

{'A': 2}
